In [1]:
import tensorflow 

import pandas as pd
import numpy as np
import os
import keras
import random
import cv2
import math
import seaborn as sns

from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Convolution2D,BatchNormalization
from tensorflow.keras.layers import Flatten,MaxPooling2D,Dropout

from tensorflow.keras.applications import VGG16, DenseNet169
from tensorflow.keras.applications.densenet import preprocess_input

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,img_to_array

from tensorflow.keras.models import Model

from tensorflow.keras.optimizers import Adam

from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

import warnings
warnings.filterwarnings("ignore")

'''Reference:
    https://www.pluralsight.com/guides/introduction-to-densenet-with-tensorflow
'''

'Reference:\n    https://www.pluralsight.com/guides/introduction-to-densenet-with-tensorflow\n'

In [2]:
print("Tensorflow-version:", tensorflow.__version__)

Tensorflow-version: 2.9.1


In [3]:
loaded_model = tensorflow.keras.models.load_model('VGG19_3_weights.h5')

In [4]:
test_df = pd.read_csv("C:/UNIFESP_Xray/Xray_small/sample_submission.csv")

test_path = "C:/UNIFESP_Xray/Xray_small_jpg/test/"

submission_data = []

for test_id in test_df['SOPInstanceUID']:
    test_img = cv2.imread(test_path+test_id+"-c.jpg")
    test_img = cv2.resize(test_img, (128,128))
    test_img = img_to_array(test_img)
    submission_data.append(test_img)

submission_data = np.array(submission_data, dtype="float32") / 255.0   #normaliztion
g3_preds = loaded_model.predict(submission_data)
print("raw preds:",g3_preds)

categories = [index for index in np.argmax(g3_preds, axis=1)]

print("argmax:",categories)

test_df['Categories'] = categories

24/24 [==============================] - 39s 2s/step
raw preds: [[3.8305880e-05 3.1367550e-03 9.9682498e-01]
 [9.0664661e-01 8.3405068e-03 8.5012972e-02]
 [4.4442106e-02 2.5206329e-03 9.5303726e-01]
 ...
 [9.5831263e-01 1.4966117e-03 4.0190715e-02]
 [5.1513925e-05 3.7689955e-04 9.9957162e-01]
 [3.9585698e-01 2.4113711e-03 6.0173166e-01]]
argmax: [2, 0, 2, 0, 0, 1, 0, 2, 0, 2, 0, 0, 0, 0, 2, 2, 0, 0, 2, 1, 0, 0, 0, 0, 0, 2, 2, 0, 2, 2, 0, 2, 0, 0, 0, 2, 0, 2, 0, 0, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 2, 0, 0, 2, 2, 2, 0, 0, 2, 2, 2, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 2, 0, 2, 0, 0, 2, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 2, 2, 0, 0, 0, 2, 0, 2, 2, 2, 1, 0, 2, 2, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 2, 0, 2, 2, 2, 0, 0, 2, 2, 1, 0, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 2, 0, 1, 0, 2, 2, 2, 0, 0, 0, 2, 2, 0, 0, 2, 0, 0, 0, 2, 0, 0, 2, 0, 0, 2, 0, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 0, 2, 1, 2, 2, 0, 2, 2, 2, 0, 0, 0, 0, 2, 0, 2, 0, 0, 1, 2, 0, 0, 2, 0, 

In [5]:
test_df

,SOPInstanceUID,Target,Categories
0,1.2.826.0.1.3680043.8.498.10001001190452685542...,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...,2
1,1.2.826.0.1.3680043.8.498.10022667601042710442...,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...,0
2,1.2.826.0.1.3680043.8.498.10024395388921105474...,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...,2
3,1.2.826.0.1.3680043.8.498.10026689165626095651...,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...,0
4,1.2.826.0.1.3680043.8.498.10035936364561920980...,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...,0
...,...,...,...
738,1.2.826.0.1.3680043.8.498.98950979295424909746...,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...,0
739,1.2.826.0.1.3680043.8.498.99120415799121180043...,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...,0
740,1.2.826.0.1.3680043.8.498.99132008951330408682...,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...,0
741,1.2.826.0.1.3680043.8.498.99611256581875030709...,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...,2


In [6]:
#Keep the single label rows
g3_index = [x for x in range(test_df.shape[0]) if test_df['Categories'][x] == 2]
print(g3_index)

[0, 2, 7, 9, 14, 15, 18, 25, 26, 28, 29, 31, 35, 37, 40, 41, 42, 45, 46, 47, 48, 49, 53, 56, 57, 58, 61, 62, 63, 64, 70, 74, 76, 78, 81, 82, 87, 93, 95, 98, 99, 103, 105, 106, 107, 110, 111, 113, 114, 115, 119, 120, 121, 122, 124, 125, 126, 129, 130, 133, 134, 135, 138, 139, 140, 141, 142, 143, 144, 145, 146, 150, 154, 155, 156, 160, 161, 164, 168, 171, 174, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 191, 192, 194, 196, 197, 199, 200, 201, 206, 208, 212, 215, 219, 220, 222, 223, 224, 227, 230, 231, 234, 235, 236, 237, 239, 241, 242, 243, 244, 245, 248, 249, 250, 254, 256, 258, 263, 264, 265, 267, 272, 275, 279, 282, 283, 285, 288, 290, 292, 293, 296, 298, 299, 301, 305, 307, 310, 311, 313, 315, 316, 319, 320, 321, 326, 329, 331, 334, 340, 341, 342, 345, 346, 347, 349, 351, 354, 356, 357, 358, 360, 361, 367, 368, 371, 372, 373, 377, 378, 380, 383, 385, 389, 390, 391, 392, 395, 396, 397, 398, 399, 401, 402, 404, 405, 408, 409, 419, 423, 433, 435, 439, 440, 443, 444, 445, 447,

In [7]:
data_g3=[]
labels=[]
# Creating Empty DataFrame and Storing it in variable df
df = pd.DataFrame()
random.seed(42)

for g3_id in test_df['SOPInstanceUID'][g3_index]:
    g3_img = cv2.imread(test_path+g3_id+"-c.jpg")
    g3_img = cv2.resize(g3_img, (128,128))
    g3_img = img_to_array(g3_img)
    data_g3.append(g3_img)

print(data_g3)
print(np.asarray(data_g3).shape)

[array([[[26., 26., 26.],
        [27., 27., 27.],
        [28., 28., 28.],
        ...,
        [46., 46., 46.],
        [50., 50., 50.],
        [60., 60., 60.]],

       [[27., 27., 27.],
        [27., 27., 27.],
        [30., 30., 30.],
        ...,
        [43., 43., 43.],
        [48., 48., 48.],
        [52., 52., 52.]],

       [[31., 31., 31.],
        [28., 28., 28.],
        [31., 31., 31.],
        ...,
        [44., 44., 44.],
        [47., 47., 47.],
        [51., 51., 51.]],

       ...,

       [[23., 23., 23.],
        [22., 22., 22.],
        [24., 24., 24.],
        ...,
        [57., 57., 57.],
        [62., 62., 62.],
        [70., 70., 70.]],

       [[22., 22., 22.],
        [25., 25., 25.],
        [25., 25., 25.],
        ...,
        [53., 53., 53.],
        [54., 54., 54.],
        [62., 62., 62.]],

       [[26., 26., 26.],
        [26., 26., 26.],
        [26., 26., 26.],
        ...,
        [53., 53., 53.],
        [50., 50., 50.],
        [60., 60., 60.]

In [8]:
data_g3 = np.array(data_g3, dtype="float32") / 255.0   #normaliztion
print(data_g3)
print(data_g3.shape)

[[[[0.10196079 0.10196079 0.10196079]
   [0.10588235 0.10588235 0.10588235]
   [0.10980392 0.10980392 0.10980392]
   ...
   [0.18039216 0.18039216 0.18039216]
   [0.19607843 0.19607843 0.19607843]
   [0.23529412 0.23529412 0.23529412]]

  [[0.10588235 0.10588235 0.10588235]
   [0.10588235 0.10588235 0.10588235]
   [0.11764706 0.11764706 0.11764706]
   ...
   [0.16862746 0.16862746 0.16862746]
   [0.1882353  0.1882353  0.1882353 ]
   [0.20392157 0.20392157 0.20392157]]

  [[0.12156863 0.12156863 0.12156863]
   [0.10980392 0.10980392 0.10980392]
   [0.12156863 0.12156863 0.12156863]
   ...
   [0.17254902 0.17254902 0.17254902]
   [0.18431373 0.18431373 0.18431373]
   [0.2        0.2        0.2       ]]

  ...

  [[0.09019608 0.09019608 0.09019608]
   [0.08627451 0.08627451 0.08627451]
   [0.09411765 0.09411765 0.09411765]
   ...
   [0.22352941 0.22352941 0.22352941]
   [0.24313726 0.24313726 0.24313726]
   [0.27450982 0.27450982 0.27450982]]

  [[0.08627451 0.08627451 0.08627451]
   [0.0

In [9]:
densenet_model = tensorflow.keras.models.load_model('DenseNet169_allData_allclass_0713-2_model.h5')

In [10]:
# Generates output predictions for the input samples. Computation is done in batches.
body_preds = densenet_model.predict(data_g3)
print(body_preds.shape)
print(body_preds)

11/11 [==============================] - 12s 908ms/step
(349, 41)
[[1.7139244e-10 2.2767228e-09 7.4763684e-10 ... 6.0678312e-10
  8.7304936e-05 9.9990952e-01]
 [4.7567582e-06 2.0619691e-06 1.8269740e-06 ... 2.0823152e-06
  6.9650924e-01 3.0071613e-01]
 [6.7219588e-07 6.2091431e-06 1.7498218e-04 ... 1.2365687e-03
  3.3044703e-06 1.1045332e-05]
 ...
 [1.2310135e-06 4.8744068e-06 8.3924093e-08 ... 6.6972272e-05
  4.5659756e-03 1.6903208e-04]
 [7.4287337e-13 1.7729974e-09 1.3465241e-10 ... 3.3836749e-11
  3.6862897e-09 1.1251140e-07]
 [9.2248389e-05 1.9221072e-05 3.3358956e-05 ... 8.7980217e-05
  2.7990482e-05 7.2969292e-06]]


In [12]:
for i in range(len(test_df)):
    if test_df['Categories'][i] == 0:
        test_df['Target'][i] = 3
    elif test_df['Categories'][i] == 1:
        test_df['Target'][i] = 14
    else:
        continue

label=['0 ','1 ','2 ','4 ','5 ','6 ','7 ','8 ','9 ','10 ','11 ','12 ','13 ','15 ','16 ','17 ','18 ','19 ','20 ','21 '] #the kaggle submission needs str in the csv
submission_labels = [label[index]+" " for index in np.argmax(body_preds, axis=1)]        
        
for j in range(len(g3_index)):
    test_df['Target'][g3_index[j]] = submission_labels[j]

test_df = test_df.drop(['Categories'], axis=1)
print(test_df)
    
test_df.to_csv("C:/Users/janic/Desktop/DSFederal/submission_VGG19_DenseNet169_7525.csv", index=False)

NameError: name 'submission_preds' is not defined

In [ ]:
print(len(test_df))